In [14]:
import pickle
import numpy as np

In [15]:
with open("../results/xu_ins/xu_ins_acc_gpt.pkl", "rb") as f:
    gpt_accs = pickle.load(f)

In [16]:
with open("../results/xu_ins/xu_ins_acc_iter.pkl", "rb") as f:
    iter_accs = pickle.load(f)

In [17]:
len(gpt_accs), len(iter_accs)

(87974, 22422)

In [5]:
gpt_accs_short = gpt_accs[:len(iter_accs)]

In [7]:
np.mean(gpt_accs_short), np.mean(iter_accs)

(0.4602502645832036, 0.4933076013198033)

In [11]:
np.mean(np.equal(gpt_accs_short, iter_accs))

0.5060075950943161

In [12]:
it_true_count = 0
true_aggreement = 0
for gpt, it in zip(gpt_accs_short, iter_accs):
    if it == 1:
        it_true_count += 1
        if gpt == 1:
            true_aggreement += 1
print(it_true_count, true_aggreement)

7924 3691


In [13]:
gpt_true_count = 0
true_aggreement = 0
for gpt, it in zip(gpt_accs_short, iter_accs):
    if gpt == 1:
        gpt_true_count += 1
        if it == 1:
            true_aggreement += 1
print(gpt_true_count, true_aggreement)

7393 3691


# Inspect dataset

In [23]:
import csv
import sys
csv.field_size_limit(sys.maxsize)


131072

In [24]:
data = []
with open("../data/xu_cluster/test_perm.tsv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for row in reader:
        data.append(row[1])

In [26]:
data = [d.split("<PUNC>") for d in data]

In [27]:
len(data)

1054

In [34]:
total = 0
for d in data:
    for i in range(len(d)):
        for j in range(len(d)):
            if i == j:
                continue
            total += 1
print(total)

906498


In [32]:
np.mean([len(d) for d in data])

21.27324478178368

In [35]:
import pickle

In [48]:
import sys
sys.path.append("../src/")
%load_ext autoreload
%autoreload 2

In [49]:
import glob
import data
import torch
from torch.utils.data import DataLoader
import training

In [50]:
ds = data.InsertionDataset(glob.glob("../data/xu/*.pkl"), 128, torch.device("cpu"))

In [51]:
len(ds.all_spans)

1054

In [56]:
dl = DataLoader(ds, batch_size=1, num_workers=0)

In [57]:
data_counter = 0
for _ in dl:
    data_counter += 1

KeyboardInterrupt: 

In [58]:
data_counter

235

In [60]:
for batch in dl:
    print(batch)
    break

[tensor([[[-6.2769e-01,  4.6157e-02,  3.5939e-01, -6.6713e-02,  5.5757e-01,
          -6.1143e-02,  1.0497e-01,  6.2606e-01,  7.4772e-02, -1.5620e-02,
          -1.2557e-01,  3.7195e-01, -2.1031e-03,  2.4489e-01, -2.1411e-01,
           1.0879e-01, -1.7785e-01, -5.7081e-02,  4.3670e-01, -2.4949e-03,
          -1.8088e-01, -1.0443e-01,  1.3801e-01,  9.1930e-02,  8.5664e-02,
           3.6354e-01, -4.5255e-01,  1.5705e-01, -3.6959e-01,  2.3285e-01,
           7.8522e-02, -1.7251e-01,  7.9905e-01,  2.4101e-01, -4.0137e-04,
          -4.3978e-01, -1.8461e-01, -1.5516e-02, -6.7140e-01,  5.7448e-01,
           5.1513e-02,  8.9472e-02,  2.7271e-01, -2.3453e-01,  1.3669e-01,
           2.8465e-01,  4.8085e-01, -3.3663e-02, -6.3398e-03, -2.8632e-02,
          -3.0595e-02,  6.8251e-02,  1.8317e-02, -3.7765e-02, -3.2800e-01,
          -2.2494e-01,  4.5574e-01,  1.6235e-02,  8.1332e-02,  2.1271e-01,
           4.0629e-01, -6.4990e-01, -1.6434e-01, -3.2075e-01,  8.5125e-02,
           3.7402e-02,  

In [65]:
batch[0].shape

torch.Size([1, 1, 768])